In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.seqsignet_full_attention_functions import seqsignet_full_attention_hyperparameter_search

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


# SeqSigNet with Attention Network

In [7]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = False

In [8]:
split_ids = torch.tensor(df_rumours['timeline_id'].astype(int))

In [9]:
num_epochs = 100
dimensions = [15]  # [50, 15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6)]#, (8, 4, 12)]
num_layers = [1]
ffn_hidden_dim_sizes = [[256,256],[512,512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [10]:
kwargs = {
    "num_epochs": num_epochs,
    "df": df_rumours,
    "id_column": "timeline_id",
    "label_column": "label",
    "embeddings": sbert_embeddings,
    "y_data": y_data,
    "output_dim": output_dim,
    "dimensions": dimensions,
    "log_signature": True,
    "swmhau_parameters": swmhau_parameters,
    "num_layers": num_layers,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "split_ids": split_ids,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# history_length=11

In [11]:
shift = 3
window_size = 5
n = 3

## UMAP

In [12]:
(
    seqsignet_attention_mha_umap_11,
    best_seqsignet_attention_mha_umap_11,
    _,
    __,
) = seqsignet_full_attention_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_mha_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_mha_umap_focal_2_3_5_3_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_mha_umap_focal_2_3_5_3_kfold_best_model.csv


In [13]:
seqsignet_attention_mha_umap_11.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2887417/32783327.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seqsignet_attention_mha_umap_11.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.684623   
                                                                         0.0005         0.691146   
                                                                         0.0010         0.692637   
                                                          (512, 512)     0.0001         0.687294   
                                                                         0.0005         0.690152   
                                                                         0.0010         0.685306   
           12              3         10        1          (256, 256)     0.0001         0.679714   
                                                                         0.0005         0.684902   
                                                                         0.0010         0.690183   
                                                          (512, 512)     0.0001         0.680584   
                                                                         0.0005         0.690276   
                                                                         0.0010         0.687108   

                                                                                              f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.656063   
                                                                         0.0005         0.662895   
                                                                         0.0010         0.656209   
                                                          (512, 512)     0.0001         0.661293   
                                                                         0.0005         0.661427   
                                                                         0.0010         0.659054   
           12              3         10        1          (256, 256)     0.0001         0.654310   
                                                                         0.0005         0.662467   
                                                                         0.0010         0.663477   
                                                          (512, 512)     0.0001         0.654509   
                                                                         0.0005         0.663058   
                                                                         0.0010         0.657701   

                                                                                        precision  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate              
15         8               4         6         1          (256, 256)     0.0001          0.655285   
                                                                         0.0005          0.662172   
                                                                         0.0010          0.662061   
                                                          (512, 512)     0.0001          0.659669   
                                                                         0.0005          0.660913   
                                                                         0.0010          0.659184   
           12              3         10        1          (256, 256)     0.0001          0.653180   
                                                                         0.0005          0.660111   
                                                                         0.0010          0.662355   
                                                          (512, 512)     0.0001          0.652981   
                                                                         0

In [14]:
best_seqsignet_attention_mha_umap_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.678658,0.655069,"[0.745271867612293, 0.5648662291771832]",0.652797,"[0.7689024390243903, 0.5366906474820143]",0.659607,"[0.7230504587155964, 0.5961640916355887]",None,0.748753,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.679963,0.658260,"[0.7443799315170462, 0.5721405432344879]",0.655843,"[0.774233508826262, 0.5374531835205992]",0.664179,"[0.716743119266055, 0.6116142781033564]",None,0.747540,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.674371,0.649766,"[0.7425961396787979, 0.5569363428861273]",0.647665,"[0.7638678387390119, 0.5314617618586641]",0.653727,"[0.7224770642201835, 0.5849760255727224]",None,0.745923,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [15]:
best_seqsignet_attention_mha_umap_11[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(512, 512)",0.1,0.0001
0,15,12,3,10,1,"(512, 512)",0.1,0.0001
0,15,12,3,10,1,"(512, 512)",0.1,0.0001


In [16]:
best_seqsignet_attention_mha_umap_11["f1"].mean()

0.6543651756843226

In [17]:
best_seqsignet_attention_mha_umap_11["precision"].mean()

0.6521015632418236

In [18]:
best_seqsignet_attention_mha_umap_11["recall"].mean()

0.6591708395855838

In [19]:
np.stack(best_seqsignet_attention_mha_umap_11["f1_scores"]).mean(axis=0)

array([0.74408265, 0.56464771])

In [20]:
np.stack(best_seqsignet_attention_mha_umap_11["precision_scores"]).mean(axis=0)

array([0.76900126, 0.53520186])

In [21]:
np.stack(best_seqsignet_attention_mha_umap_11["recall_scores"]).mean(axis=0)

array([0.72075688, 0.5975848 ])

## Random Projections

In [22]:
(
    seqsignet_attention_mha_grp_11,
    best_seqsignet_attention_mha_grp_11,
    _,
    __,
) = seqsignet_full_attention_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/seqsignet_attention_mha_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_mha_grp_focal_2_3_5_3_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_mha_grp_focal_2_3_5_3_kfold_best_model.csv


In [23]:
seqsignet_attention_mha_grp_11.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2887417/2541425521.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seqsignet_attention_mha_grp_11.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.684809   
                                                                                      0.0005         0.690214   
                                                                                      0.0010         0.674309   
                                                                         0.2          0.0001         0.681454   
                                                                                      0.0005         0.695061   
                                                                                      0.0010         0.701522   
                                                          (512, 512)     0.1          0.0001         0.683504   
                                                                                      0.0005         0.688164   
                                                                                      0.0010         0.689034   
                                                                         0.2          0.0001         0.684560   
                                                                                      0.0005         0.691643   
                                                                                      0.0010         0.691022   
           12              3         10        1          (256, 256)     0.1          0.0001         0.669897   
                                                                                      0.0005         0.682510   
                                                                                      0.0010         0.685865   
                                                                         0.2          0.0001         0.671761   
                                                                                      0.0005         0.687543   
                                                                                      0.0010         0.693942   
                                                          (512, 512)     0.1          0.0001         0.677664   
                                                                                      0.0005         0.680646   
                                                                                      0.0010         0.678037   
                                                                         0.2          0.0001         0.675116   
                                                                                      0.0005         0.685617   
                                                                                      0.0010         0.683815   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.662651   
                                                                                      0.0005         0.663641   
                                                                                      0.0010         0.636675   
                                                                         0.2          0.0001         0.662406   
                                                                                      0.0005         0.671529   
                                                                                      0.0010         0.670616   
                                                          (512, 512)     0.1          0.0001         0.667214   
                                                                                      0.0005         0.666976   
                                

In [24]:
best_seqsignet_attention_mha_grp_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.674930,0.659360,"[0.7321867321867321, 0.586533902323376]",0.658383,"[0.7883597883597884, 0.5284066638188808]",0.671258,"[0.6834862385321101, 0.6590303676078849]",None,0.767893,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.681640,0.662647,"[0.7426935824043387, 0.5826001955034213]",0.660344,"[0.7825396825396825, 0.5381489841986457]",0.670882,"[0.7067087155963303, 0.6350559403303143]",None,0.773150,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.673439,0.658363,"[0.7301293900184842, 0.5865974516281266]",0.657710,"[0.788948069241012, 0.5264718339686574]",0.670850,"[0.6794724770642202, 0.662226957911561]",None,0.759806,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [26]:
best_seqsignet_attention_mha_grp_11[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(512, 512)",0.2,0.0001
0,15,12,3,10,1,"(512, 512)",0.2,0.0001
0,15,12,3,10,1,"(512, 512)",0.2,0.0001


In [27]:
best_seqsignet_attention_mha_grp_11["f1"].mean()

0.6601235423440798

In [28]:
best_seqsignet_attention_mha_grp_11["precision"].mean()

0.6588125036877778

In [29]:
best_seqsignet_attention_mha_grp_11["recall"].mean()

0.6709967828404034

In [30]:
np.stack(best_seqsignet_attention_mha_grp_11["f1_scores"]).mean(axis=0)

array([0.73500323, 0.58524385])

In [31]:
np.stack(best_seqsignet_attention_mha_grp_11["precision_scores"]).mean(axis=0)

array([0.78661585, 0.53100916])

In [32]:
np.stack(best_seqsignet_attention_mha_grp_11["recall_scores"]).mean(axis=0)

array([0.68988914, 0.65210442])

# history_length=20

In [33]:
shift = 3
window_size = 5
n = 6

## UMAP

In [34]:
(
    seqsignet_attention_mha_umap_20,
    best_seqsignet_attention_mha_umap_20,
    _,
    __,
) = seqsignet_full_attention_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_mha_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_mha_umap_focal_2_3_5_6_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_mha_umap_focal_2_3_5_6_kfold_best_model.csv


In [35]:
seqsignet_attention_mha_umap_20.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2887417/1243729295.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seqsignet_attention_mha_umap_20.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.678503   
                                                                         0.0005         0.690898   
                                                                         0.0010         0.688288   
                                                          (512, 512)     0.0001         0.679279   
                                                                         0.0005         0.691612   
                                                                         0.0010         0.691674   
           12              3         10        1          (256, 256)     0.0001         0.677074   
                                                                         0.0005         0.671575   
                                                                         0.0010         0.679186   
                                                          (512, 512)     0.0001         0.668903   
                                                                         0.0005         0.685337   
                                                                         0.0010         0.684809   

                                                                                              f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.654680   
                                                                         0.0005         0.663172   
                                                                         0.0010         0.656955   
                                                          (512, 512)     0.0001         0.654502   
                                                                         0.0005         0.670636   
                                                                         0.0010         0.655905   
           12              3         10        1          (256, 256)     0.0001         0.653874   
                                                                         0.0005         0.649593   
                                                                         0.0010         0.657913   
                                                          (512, 512)     0.0001         0.647047   
                                                                         0.0005         0.667042   
                                                                         0.0010         0.658539   

                                                                                        precision  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate              
15         8               4         6         1          (256, 256)     0.0001          0.652657   
                                                                         0.0005          0.662529   
                                                                         0.0010          0.660503   
                                                          (512, 512)     0.0001          0.652607   
                                                                         0.0005          0.668751   
                                                                         0.0010          0.661443   
           12              3         10        1          (256, 256)     0.0001          0.652168   
                                                                         0.0005          0.648876   
                                                                         0.0010          0.655794   
                                                          (512, 512)     0.0001          0.645120   
                                                                         0

In [36]:
best_seqsignet_attention_mha_umap_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.668034,0.638056,"[0.742220292372268, 0.5338916514001569]",0.637090,"[0.749488453668518, 0.5246913580246914]",0.639256,"[0.7350917431192661, 0.5434203516249334]",None,0.831783,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.675862,0.652538,"[0.7425610658771281, 0.5625157232704403]",0.650283,"[0.7676767676767676, 0.5328884652049571]",0.657334,"[0.7190366972477065, 0.595631326584976]",None,0.837984,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.663374,0.629647,"[0.7414089347079037, 0.5178857447944474]",0.629777,"[0.7405606407322655, 0.5189941144997324]",0.629521,"[0.7422591743119266, 0.5167820990942994]",None,0.825044,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [37]:
best_seqsignet_attention_mha_umap_20[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001


In [38]:
best_seqsignet_attention_mha_umap_20["f1"].mean()

0.6400805687370573

In [39]:
best_seqsignet_attention_mha_umap_20["precision"].mean()

0.6390499666344887

In [40]:
best_seqsignet_attention_mha_umap_20["recall"].mean()

0.6420368986638513

In [41]:
np.stack(best_seqsignet_attention_mha_umap_20["f1_scores"]).mean(axis=0)

array([0.74206343, 0.53809771])

In [42]:
np.stack(best_seqsignet_attention_mha_umap_20["precision_scores"]).mean(axis=0)

array([0.75257529, 0.52552465])

In [43]:
np.stack(best_seqsignet_attention_mha_umap_20["recall_scores"]).mean(axis=0)

array([0.7321292 , 0.55194459])

## Random Projections

In [44]:
(
    seqsignet_attention_mha_grp_20,
    best_seqsignet_attention_mha_grp_20,
    _,
    __,
) = seqsignet_full_attention_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/seqsignet_attention_mha_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_mha_grp_focal_2_3_5_6_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_mha_grp_focal_2_3_5_6_kfold_best_model.csv


In [45]:
seqsignet_attention_mha_grp_20.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2887417/910681948.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seqsignet_attention_mha_grp_20.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.681019   
                                                                                      0.0005         0.685492   
                                                                                      0.0010         0.683815   
                                                                         0.2          0.0001         0.678161   
                                                                                      0.0005         0.675241   
                                                                                      0.0010         0.689469   
                                                          (512, 512)     0.1          0.0001         0.677788   
                                                                                      0.0005         0.680708   
                                                                                      0.0010         0.686114   
                                                                         0.2          0.0001         0.686176   
                                                                                      0.0005         0.672010   
                                                                                      0.0010         0.692948   
           12              3         10        1          (256, 256)     0.1          0.0001         0.665610   
                                                                                      0.0005         0.679963   
                                                                                      0.0010         0.677664   
                                                                         0.2          0.0001         0.666418   
                                                                                      0.0005         0.682945   
                                                                                      0.0010         0.681454   
                                                          (512, 512)     0.1          0.0001         0.678596   
                                                                                      0.0005         0.676235   
                                                                                      0.0010         0.674433   
                                                                         0.2          0.0001         0.681454   
                                                                                      0.0005         0.670581   
                                                                                      0.0010         0.672321   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.658516   
                                                                                      0.0005         0.666930   
                                                                                      0.0010         0.660708   
                                                                         0.2          0.0001         0.659291   
                                                                                      0.0005         0.656236   
                                                                                      0.0010         0.655266   
                                                          (512, 512)     0.1          0.0001         0.655299   
                                                                                      0.0005         0.661752   
                                

In [46]:
best_seqsignet_attention_mha_grp_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.687978,0.664007,"[0.7537511032656663, 0.574262461851475]",0.661705,"[0.7740181268882175, 0.5493917274939173]",0.668005,"[0.7345183486238532, 0.6014917421417155]",None,0.835018,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.668220,0.643490,"[0.7373856594865741, 0.5495951417004049]",0.641474,"[0.7595744680851064, 0.5233734939759036]",0.647520,"[0.7164564220183486, 0.5785828449653703]",None,0.836231,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.674744,0.652826,"[0.7400566065842394, 0.565596216081653]",0.650537,"[0.7702325581395348, 0.5308411214953271]",0.658689,"[0.7121559633027523, 0.6052210974960043]",None,0.839601,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [48]:
best_seqsignet_attention_mha_grp_20[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(256, 256)",0.2,0.0001
0,15,12,3,10,1,"(256, 256)",0.2,0.0001
0,15,12,3,10,1,"(256, 256)",0.2,0.0001


In [49]:
best_seqsignet_attention_mha_grp_20["f1"].mean()

0.6534411981616689

In [50]:
best_seqsignet_attention_mha_grp_20["precision"].mean()

0.6512385826796677

In [51]:
best_seqsignet_attention_mha_grp_20["recall"].mean()

0.6580710697580073

In [52]:
np.stack(best_seqsignet_attention_mha_grp_20["f1_scores"]).mean(axis=0)

array([0.74373112, 0.56315127])

In [53]:
np.stack(best_seqsignet_attention_mha_grp_20["precision_scores"]).mean(axis=0)

array([0.76794172, 0.53453545])

In [54]:
np.stack(best_seqsignet_attention_mha_grp_20["recall_scores"]).mean(axis=0)

array([0.72104358, 0.59509856])

# history_length=35

In [11]:
shift = 3
window_size = 5
n = 11

## UMAP

In [56]:
(
    seqsignet_attention_mha_umap_35,
    best_seqsignet_attention_mha_umap_35,
    _,
    __,
) = seqsignet_full_attention_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_mha_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_mha_umap_focal_2_3_5_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_mha_umap_focal_2_3_5_11_kfold_best_model.csv


In [57]:
seqsignet_attention_mha_umap_35.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2887417/551868984.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seqsignet_attention_mha_umap_35.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.673998   
                                                                         0.0005         0.686797   
                                                                         0.0010         0.699783   
                                                          (512, 512)     0.0001         0.687667   
                                                                         0.0005         0.693538   
                                                                         0.0010         0.682696   
           12              3         10        1          (256, 256)     0.0001         0.672227   
                                                                         0.0005         0.680522   
                                                                         0.0010         0.695340   
                                                          (512, 512)     0.0001         0.683877   
                                                                         0.0005         0.669494   
                                                                         0.0010         0.687574   

                                                                                              f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.643795   
                                                                         0.0005         0.662772   
                                                                         0.0010         0.668011   
                                                          (512, 512)     0.0001         0.659811   
                                                                         0.0005         0.667580   
                                                                         0.0010         0.651136   
           12              3         10        1          (256, 256)     0.0001         0.646420   
                                                                         0.0005         0.656598   
                                                                         0.0010         0.673986   
                                                          (512, 512)     0.0001         0.658474   
                                                                         0.0005         0.649316   
                                                                         0.0010         0.665604   

                                                                                        precision  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate              
15         8               4         6         1          (256, 256)     0.0001          0.643346   
                                                                         0.0005          0.660955   
                                                                         0.0010          0.673269   
                                                          (512, 512)     0.0001          0.658613   
                                                                         0.0005          0.666572   
                                                                         0.0010          0.652047   
           12              3         10        1          (256, 256)     0.0001          0.645133   
                                                                         0.0005          0.654807   
                                                                         0.0010          0.671623   
                                                          (512, 512)     0.0001          0.656661   
                                                                         0

In [58]:
best_seqsignet_attention_mha_umap_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.665797,0.645491,"[0.7303353887802676, 0.5606469002695419]",0.643584,"[0.768111357165454, 0.5190562613430127]",0.652792,"[0.6961009174311926, 0.6094832179009056]",None,0.864807,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.693383,0.667231,"[0.76051827049061, 0.573944573944574]",0.665510,"[0.7725524992605738, 0.5584677419354839]",0.669578,"[0.7488532110091743, 0.5903036760788493]",None,0.872759,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.677353,0.647051,"[0.7504685022343952, 0.5436330081729502]",0.646406,"[0.7547115105827776, 0.5381002087682673]",0.647777,"[0.7462729357798165, 0.5492807671816728]",None,0.854832,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [59]:
best_seqsignet_attention_mha_umap_35[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(256, 256)",0.2,0.0001
0,15,12,3,10,1,"(256, 256)",0.2,0.0001
0,15,12,3,10,1,"(256, 256)",0.2,0.0001


In [60]:
best_seqsignet_attention_mha_umap_35["f1"].mean()

0.6532577739820565

In [61]:
best_seqsignet_attention_mha_umap_35["precision"].mean()

0.6518332631759282

In [62]:
best_seqsignet_attention_mha_umap_35["recall"].mean()

0.6567157875636018

In [63]:
np.stack(best_seqsignet_attention_mha_umap_35["f1_scores"]).mean(axis=0)

array([0.74710739, 0.55940816])

In [64]:
np.stack(best_seqsignet_attention_mha_umap_35["precision_scores"]).mean(axis=0)

array([0.76512512, 0.5385414 ])

In [65]:
np.stack(best_seqsignet_attention_mha_umap_35["recall_scores"]).mean(axis=0)

array([0.73040902, 0.58302255])

## Random Projections

In [12]:
(
    seqsignet_attention_mha_grp_35,
    best_seqsignet_attention_mha_grp_35,
    _,
    __,
) = seqsignet_full_attention_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/seqsignet_attention_mha_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_mha_grp_focal_2_3_5_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_mha_grp_focal_2_3_5_11_kfold_best_model.csv


In [13]:
seqsignet_attention_mha_grp_35.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_570758/4029474570.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seqsignet_attention_mha_grp_35.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.665424   
                                                                                      0.0005         0.674992   
                                                                                      0.0010         0.669525   
                                                                         0.2          0.0001         0.672818   
                                                                                      0.0005         0.689407   
                                                                                      0.0010         0.703635   
                                                          (512, 512)     0.1          0.0001         0.670829   
                                                                                      0.0005         0.679217   
                                                                                      0.0010         0.697919   
                                                                         0.2          0.0001         0.677291   
                                                                                      0.0005         0.686797   
                                                                                      0.0010         0.687232   
           12              3         10        1          (256, 256)     0.1          0.0001         0.670146   
                                                                                      0.0005         0.685679   
                                                                                      0.0010         0.677788   
                                                                         0.2          0.0001         0.684250   
                                                                                      0.0005         0.683566   
                                                                                      0.0010         0.677975   
                                                          (512, 512)     0.1          0.0001         0.681454   
                                                                                      0.0005         0.667412   
                                                                                      0.0010         0.684436   
                                                                         0.2          0.0001         0.680895   
                                                                                      0.0005         0.673750   
                                                                                      0.0010         0.670457   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.639978   
                                                                                      0.0005         0.654054   
                                                                                      0.0010         0.644499   
                                                                         0.2          0.0001         0.641611   
                                                                                      0.0005         0.665550   
                                                                                      0.0010         0.669486   
                                                          (512, 512)     0.1          0.0001         0.645542   
                                                                                      0.0005         0.649455   
                                

In [14]:
best_seqsignet_attention_mha_grp_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.664678,0.636653,"[0.737563822027717, 0.535741935483871]",0.635168,"[0.7508167508167508, 0.5195195195195195]",0.638890,"[0.7247706422018348, 0.5530101225359616]",None,0.870198,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.665797,0.637783,"[0.7385153857372029, 0.5370513813581203]",0.636300,"[0.7515583259127337, 0.5210420841683366]",0.639997,"[0.7259174311926605, 0.554075652637187]",None,0.872220,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.701584,0.672131,"[0.7704001147282376, 0.5738621240351345]",0.672068,"[0.7707317073170732, 0.573404255319149]",0.672195,"[0.7700688073394495, 0.5743207245604688]",None,0.860628,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [15]:
best_seqsignet_attention_mha_grp_35[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001


In [16]:
best_seqsignet_attention_mha_grp_35["f1"].mean()

0.6488557938950472

In [17]:
best_seqsignet_attention_mha_grp_35["precision"].mean()

0.647845440508927

In [18]:
best_seqsignet_attention_mha_grp_35["recall"].mean()

0.6503605634112605

In [19]:
np.stack(best_seqsignet_attention_mha_grp_35["f1_scores"]).mean(axis=0)

array([0.74882644, 0.54888515])

In [20]:
np.stack(best_seqsignet_attention_mha_grp_35["precision_scores"]).mean(axis=0)

array([0.75770226, 0.53798862])

In [21]:
np.stack(best_seqsignet_attention_mha_grp_35["recall_scores"]).mean(axis=0)

array([0.74025229, 0.56046883])

# history_length=80

In [22]:
shift = 3
window_size = 5
n = 26

## UMAP

In [23]:
(
    seqsignet_attention_mha_umap_80,
    best_seqsignet_attention_mha_umap_80,
    _,
    __,
) = seqsignet_full_attention_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_mha_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 26: history length = 80
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 26: history length = 80
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_mha_umap_focal_2_3_5_26_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_mha_umap_focal_2_3_5_26_kfold_best_model.csv


In [24]:
seqsignet_attention_mha_umap_80.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_570758/2278199386.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seqsignet_attention_mha_umap_80.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.686269   
                                                                         0.0005         0.680118   
                                                                         0.0010         0.680615   
                                                          (512, 512)     0.0001         0.693632   
                                                                         0.0005         0.688195   
                                                                         0.0010         0.688506   
           12              3         10        1          (256, 256)     0.0001         0.684498   
                                                                         0.0005         0.691426   
                                                                         0.0010         0.686269   
                                                          (512, 512)     0.0001         0.687853   
                                                                         0.0005         0.675458   
                                                                         0.0010         0.691395   

                                                                                              f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.652794   
                                                                         0.0005         0.651833   
                                                                         0.0010         0.627607   
                                                          (512, 512)     0.0001         0.661797   
                                                                         0.0005         0.658409   
                                                                         0.0010         0.639366   
           12              3         10        1          (256, 256)     0.0001         0.656132   
                                                                         0.0005         0.664782   
                                                                         0.0010         0.657607   
                                                          (512, 512)     0.0001         0.661443   
                                                                         0.0005         0.652837   
                                                                         0.0010         0.665694   

                                                                                        precision  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate              
15         8               4         6         1          (256, 256)     0.0001          0.654484   
                                                                         0.0005          0.651241   
                                                                         0.0010          0.640547   
                                                          (512, 512)     0.0001          0.662695   
                                                                         0.0005          0.658192   
                                                                         0.0010          0.653413   
           12              3         10        1          (256, 256)     0.0001          0.655727   
                                                                         0.0005          0.663410   
                                                                         0.0010          0.656990   
                                                          (512, 512)     0.0001          0.659789   
                                                                         0

In [25]:
best_seqsignet_attention_mha_umap_80

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.680149,0.658834,"[0.7441097524604832, 0.5735586481113321]",0.656410,"[0.7753262896208826, 0.5374941779226828]",0.665060,"[0.7153096330275229, 0.6148108684070325]",None,0.872355,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.684809,0.660485,"[0.7513600941038084, 0.5696105879358615]",0.658263,"[0.7712043465137338, 0.5453216374269005]",0.664338,"[0.7325114678899083, 0.5961640916355887]",None,0.870198,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.689469,0.650228,"[0.7673834124546216, 0.5330717488789238]",0.655174,"[0.7479586281981492, 0.5623891188645772]",0.647252,"[0.7878440366972477, 0.5066595631326585]",None,0.862515,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [26]:
best_seqsignet_attention_mha_umap_80[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001


In [27]:
best_seqsignet_attention_mha_umap_80["f1"].mean()

0.6565157073241717

In [28]:
best_seqsignet_attention_mha_umap_80["precision"].mean()

0.6566156997578211

In [29]:
best_seqsignet_attention_mha_umap_80["recall"].mean()

0.6588832767983265

In [30]:
np.stack(best_seqsignet_attention_mha_umap_80["f1_scores"]).mean(axis=0)

array([0.75428442, 0.55874699])

In [31]:
np.stack(best_seqsignet_attention_mha_umap_80["precision_scores"]).mean(axis=0)

array([0.76482975, 0.54840164])

In [32]:
np.stack(best_seqsignet_attention_mha_umap_80["recall_scores"]).mean(axis=0)

array([0.74522171, 0.57254484])

## Random Projections

In [33]:
(
    seqsignet_attention_mha_grp_80,
    best_seqsignet_attention_mha_grp_80,
    _,
    __,
) = seqsignet_full_attention_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/seqsignet_attention_mha_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
given shift 3, window size 5 and n 26: history length = 80
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 26: history length = 80
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_mha_grp_focal_2_3_5_26_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_mha_grp_focal_2_3_5_26_kfold_best_model.csv


In [34]:
seqsignet_attention_mha_grp_80.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_570758/22412081.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seqsignet_attention_mha_grp_80.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.682262   
                                                                                      0.0005         0.702765   
                                                                                      0.0010         0.692389   
                                                                         0.2          0.0001         0.687791   
                                                                                      0.0005         0.704877   
                                                                                      0.0010         0.694564   
                                                          (512, 512)     0.1          0.0001         0.690090   
                                                                                      0.0005         0.709972   
                                                                                      0.0010         0.674619   
                                                                         0.2          0.0001         0.684871   
                                                                                      0.0005         0.693569   
                                                                                      0.0010         0.700777   
           12              3         10        1          (256, 256)     0.1          0.0001         0.682386   
                                                                                      0.0005         0.678285   
                                                                                      0.0010         0.684188   
                                                                         0.2          0.0001         0.683504   
                                                                                      0.0005         0.679279   
                                                                                      0.0010         0.687108   
                                                          (512, 512)     0.1          0.0001         0.694253   
                                                                                      0.0005         0.681330   
                                                                                      0.0010         0.693259   
                                                                         0.2          0.0001         0.696614   
                                                                                      0.0005         0.682759   
                                                                                      0.0010         0.692948   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.652778   
                                                                                      0.0005         0.679254   
                                                                                      0.0010         0.649381   
                                                                         0.2          0.0001         0.658504   
                                                                                      0.0005         0.672375   
                                                                                      0.0010         0.642369   
                                                          (512, 512)     0.1          0.0001         0.655230   
                                                                                      0.0005         0.680619   
                                

In [35]:
best_seqsignet_attention_mha_grp_80

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.690401,0.668669,"[0.753524261759905, 0.5838135805562515]",0.666039,"[0.78099046447247, 0.5510879848628193]",0.674298,"[0.7279243119266054, 0.620671283963772]",None,0.894865,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.705499,0.677955,"[0.7721372944909143, 0.58377239199157]",0.677113,"[0.7768427161926872, 0.5773840541948931]",0.678896,"[0.7674885321100917, 0.5903036760788493]",None,0.887856,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.676048,0.639560,"[0.7542420814479638, 0.5248769819573538]",0.641582,"[0.744140625, 0.5390230207748455]",0.638038,"[0.7646215596330275, 0.5114544485881726]",None,0.876533,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [36]:
best_seqsignet_attention_mha_grp_80[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(256, 256)",0.2,0.0005
0,15,12,3,10,1,"(256, 256)",0.2,0.0005
0,15,12,3,10,1,"(256, 256)",0.2,0.0005


In [37]:
best_seqsignet_attention_mha_grp_80["f1"].mean()

0.6620610987006598

In [38]:
best_seqsignet_attention_mha_grp_80["precision"].mean()

0.6615781442496192

In [39]:
best_seqsignet_attention_mha_grp_80["recall"].mean()

0.6637439687167531

In [40]:
np.stack(best_seqsignet_attention_mha_grp_80["f1_scores"]).mean(axis=0)

array([0.75996788, 0.56415432])

In [41]:
np.stack(best_seqsignet_attention_mha_grp_80["precision_scores"]).mean(axis=0)

array([0.7673246 , 0.55583169])

In [42]:
np.stack(best_seqsignet_attention_mha_grp_80["recall_scores"]).mean(axis=0)

array([0.7533448 , 0.57414314])